## 중요: 출력이 두 번씩 나오는 경우
만약 출력이 중복되어 나타난다면:
1. **Kernel > Restart Kernel...** 선택
2. 모든 셀을 처음부터 순서대로 다시 실행
3. 캐시된 출력과 변수 상태가 초기화됩니다

In [1]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-cohere langchain-upstage langchain-google-genai langchain-huggingface pypdf python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'false'  # 트레이싱 비활성화
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('langchain_api_key', '')
os.environ["LANGCHAIN_PROJECT"] = 'default'
os.environ['GOOGLE_API_KEY'] = os.getenv('gemini_api_key', '')
os.environ['OPENAI_API_KEY'] = os.getenv('openai_api_key', '')
# os.environ["UPSTAGE_API_KEY"] = os.getenv('upstage_api_key', '')
# os.environ["PPLX_API_KEY"] = os.getenv('pplx_api_key', '')
# os.environ["COHERE_API_KEY"] = os.getenv('cohere_api_key', '')


Part 1: Overview

In [18]:
!pip install sentence-transformers

E0000 00:00:1762161931.001135  481661 backup_poller.cc:138] Run client channel backup poller: UNKNOWN:pollset_work {children:[UNKNOWN:epoll_wait: Bad file descriptor (9)]}
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [33]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_upstage import ChatUpstage
from langchain_upstage import UpstageEmbeddings

import os
import getpass

#### INDEXING ####

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

from langchain_community.document_loaders import PyPDFLoader
import glob

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=100)

pdf_files = glob.glob("pdf/SnT GPS_*.pdf")
print(f"Found {len(pdf_files)} PDF files")

all_docs = []
for pdf_file in pdf_files:
    try:
        loader = PyPDFLoader(pdf_file, extract_images=False)
        docs = loader.load()
        # 메타데이터 정리: unhashable 타입 처리
        for doc in docs:
            cleaned_metadata = {}
            for k, v in doc.metadata.items():
                try:
                    if isinstance(v, (str, int, float, bool)):
                        cleaned_metadata[k] = v
                    else:
                        cleaned_metadata[k] = str(v)
                except Exception:
                    cleaned_metadata[k] = "unable_to_convert"
            doc.metadata = cleaned_metadata
        all_docs.extend(docs)
        print(f"✓ Loaded {pdf_file}: {len(docs)} pages")
    except Exception as e:
        print(f"✗ Error loading {pdf_file}: {e}")

print(f"Total documents loaded: {len(all_docs)}")

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(all_docs)
print(f"Total splits: {len(splits)}")

# Embed
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings,
    collection_name="rag_collection"
)
retriever = vectorstore.as_retriever()

from langsmith import Client
client = Client(api_key=os.environ['LANGCHAIN_API_KEY'])
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

#### RETRIEVAL and GENERATION ####

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(all_docs)
print(f"Total splits: {len(splits)}")

# Embed
print("▷ Vectorstore 생성 중...")
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings,
    collection_name="rag_collection"
)
retriever = vectorstore.as_retriever()
print("✓ Vectorstore 생성 완료.")

from langsmith import Client
client = Client(api_key=os.environ['LANGCHAIN_API_KEY'])
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)
print("✓ Langsmith 프롬프트 로드 완료.")


#### RETRIEVAL and GENERATION ####

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print("\n✓ 모든 설정이 완료되었습니다.")

Found 10 PDF files
✓ Loaded pdf/SnT GPS_287호_최종.pdf: 70 pages
✓ Loaded pdf/SnT GPS_287호_최종.pdf: 70 pages
✗ Error loading pdf/SnT GPS_290호_최종본.pdf: unhashable type: 'ArrayObject'
✗ Error loading pdf/SnT GPS_290호_최종본.pdf: unhashable type: 'ArrayObject'
✓ Loaded pdf/SnT GPS_286호_최종.pdf: 68 pages
✓ Loaded pdf/SnT GPS_286호_최종.pdf: 68 pages
✗ Error loading pdf/SnT GPS_288호_최종.pdf: unhashable type: 'ArrayObject'
✗ Error loading pdf/SnT GPS_288호_최종.pdf: unhashable type: 'ArrayObject'
✗ Error loading pdf/SnT GPS_285호_최종.pdf: unhashable type: 'ArrayObject'
✗ Error loading pdf/SnT GPS_285호_최종.pdf: unhashable type: 'ArrayObject'
✓ Loaded pdf/SnT GPS_292호_최종.pdf: 69 pages
✓ Loaded pdf/SnT GPS_292호_최종.pdf: 69 pages
✗ Error loading pdf/SnT GPS_283호_최종.pdf: unhashable type: 'ArrayObject'
✗ Error loading pdf/SnT GPS_283호_최종.pdf: unhashable type: 'ArrayObject'
✓ Loaded pdf/SnT GPS_284호_최종.pdf: 80 pages
✓ Loaded pdf/SnT GPS_284호_최종.pdf: 80 pages
✓ Loaded pdf/SnT GPS_291호_최종.pdf: 70 pages
✓ Loaded pdf/SnT

In [37]:
pip install pypdfium2

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.0 MB ? eta -:--:--Downloading pypdfium2-5.0.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
import os
import glob
import getpass
from typing import Any, Dict, List

# ===== Embeddings =====
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
try:
    from langchain_community.document_loaders import PyPDFium2Loader
except Exception:
    PyPDFium2Loader = None

# ===== LangChain core =====
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# ===== ENV =====
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# ===== JSON-safe 변환기 =====
def to_json_safe(obj: Any) -> Any:
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj
    if isinstance(obj, (list, tuple, set)):
        return [to_json_safe(x) for x in obj]
    if isinstance(obj, dict):
        return {str(k): to_json_safe(v) for k, v in obj.items()}
    try:
        import json
        json.dumps(obj)
        return obj
    except Exception:
        return str(obj)

def rebuild_safe_documents(docs: List[Document], source_path: str) -> List[Document]:
    safe: List[Document] = []
    import os as _os
    fname = _os.path.basename(source_path)
    for d in docs:
        page_num = d.metadata.get("page", d.metadata.get("page_number"))
        safe_meta: Dict[str, Any] = {
            "source": source_path,
            "file_name": fname,
            "page": int(page_num) if page_num is not None else None,
        }
        # 원본 메타데이터 추가 보존(충돌 시 safe_meta 우선)
        extra = {k: v for k, v in d.metadata.items() if k not in ("source", "file_name", "page")}
        safe_meta.update(to_json_safe(extra))
        safe.append(Document(page_content=d.page_content, metadata=safe_meta))
    return safe

# ===== 로더 폴백 체인 =====
def load_with_fallback(pdf_path: str) -> List[Document]:
    errors = []

    # 1) PyPDFLoader (가벼움, 그러나 pypdf 메타 이슈 발생 가능)
    try:
        docs = PyPDFLoader(pdf_path, extract_images=False).load()
        return docs
    except Exception as e:
        errors.append(("PyPDFLoader", str(e)))

    # 2) PyPDFium2Loader (텍스트 안정적, 메타잡음 적음)
    if PyPDFium2Loader is not None:
        try:
            docs = PyPDFium2Loader(pdf_path).load()
            return docs
        except Exception as e:
            errors.append(("PyPDFium2Loader", str(e)))


    # 전부 실패
    raise RuntimeError(" all loaders failed: " + " | ".join([f"{n}: {m}" for n, m in errors]))

# ===== 메인 =====
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=100)

# 중복 제거 및 정렬
pdf_files = sorted(set(glob.glob("pdf/SnT GPS_*.pdf")))
print(f"Found {len(pdf_files)} unique PDF files")

all_docs: List[Document] = []
ok, fail = 0, 0

for pdf_file in pdf_files:
    try:
        raw_docs = load_with_fallback(pdf_file)  # 여기서 로더 내부 오류를 흡수
        safe_docs = rebuild_safe_documents(raw_docs, pdf_file)
        all_docs.extend(safe_docs)
        ok += 1
        print(f"✓ {os.path.basename(pdf_file)}: {len(safe_docs)} pages (sanitized)")
    except Exception as e:
        fail += 1
        print(f"✗ {os.path.basename(pdf_file)}: {e}")

print(f"Load summary: success {ok}, failed {fail}")
print(f"Total documents loaded: {len(all_docs)}")

# ===== Split =====
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(all_docs)
print(f"Total splits: {len(splits)}")

# ===== Vector store =====
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="rag_collection",
    # persist_directory="chroma_db",
)
retriever = vectorstore.as_retriever()

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

print("Indexing complete.")


Found 10 unique PDF files
✓ SnT GPS_283호_최종.pdf: 66 pages (sanitized)
✓ SnT GPS_284호_최종.pdf: 80 pages (sanitized)
✓ SnT GPS_285호_최종.pdf: 68 pages (sanitized)
✓ SnT GPS_286호_최종.pdf: 68 pages (sanitized)
✓ SnT GPS_287호_최종.pdf: 70 pages (sanitized)
✓ SnT GPS_288호_최종.pdf: 67 pages (sanitized)
✓ SnT GPS_289호_최종.pdf: 67 pages (sanitized)
✓ SnT GPS_290호_최종본.pdf: 73 pages (sanitized)
✓ SnT GPS_291호_최종.pdf: 70 pages (sanitized)
✓ SnT GPS_292호_최종.pdf: 69 pages (sanitized)
Load summary: success 10, failed 0
Total documents loaded: 698
Total splits: 1258
Indexing complete.


## 3개 LLM 비교 분석

In [ ]:
# ===== Prompt: LangSmith에서 모델 없이 가져오기 =====
from langsmith import Client
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

import os

client = Client(api_key=os.environ.get("LANGCHAIN_API_KEY"))
# 중요: include_model=False 로 모델 분리
prompt = client.pull_prompt("rlm/rag-prompt", include_model=False)

# retriever 검색 폭
try:
    retriever.search_kwargs = {"k": 10}
except Exception:
    pass

# ===== LLM 모델 설정 =====
llm_models = {
    "GPT-3.5-turbo": ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=None),
    "GPT-4o-mini": ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=None),
    "Gemini-2.0-flash": ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0, max_output_tokens=None)
}

# ===== RAG 체인 생성 =====
rag_chains = {}
for model_name, llm in llm_models.items():
    chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    rag_chains[model_name] = chain


In [41]:
# ===== 질문 리스트 =====
questions = [
    "AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년에 얼마로 성장하나요?",
    "연방하원 선거 45일 후, 기독민주당(CDU)/기독사회당(CSU) 연합과 사민당(SPD)은 어떤 문서에 연정 합의했나요?",
    "BCG는 각국 정부가 직면한 데이터 주권 및 보안 문제를 해결하고자 어떤 보고서를 발표했나요?"
]
print(questions)

['AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년에 얼마로 성장하나요?', '연방하원 선거 45일 후, 기독민주당(CDU)/기독사회당(CSU) 연합과 사민당(SPD)은 어떤 문서에 연정 합의했나요?', 'BCG는 각국 정부가 직면한 데이터 주권 및 보안 문제를 해결하고자 어떤 보고서를 발표했나요?']


In [44]:
# 각 질문에 대해 관련 문서 검색
for idx, question in enumerate(questions, 1):
    print(f"\n{'='*100}")
    print(f"Question {idx}: {question}")
    print(f"{'='*100}\n")
    
    # 관련 문서 검색
    relevant_docs = retriever.invoke(question)
    print(f"[검색된 관련 문서 정보]")
    print(f"총 {len(relevant_docs)}개의 문서 청크가 검색되었습니다.")
    for i, doc in enumerate(relevant_docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'Unknown')
        print(f"  - 문서 {i}: {source} (페이지: {page})")
        print(f"    내용 미리보기: {doc.page_content[:150]}...")
    print()



Question 1: AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년에 얼마로 성장하나요?

[검색된 관련 문서 정보]
총 4개의 문서 청크가 검색되었습니다.
  - 문서 1: pdf/SnT GPS_290호_최종본.pdf (페이지: 3)
    내용 미리보기: 2
피지컬 AI에 대한 시장 기대도 꾸준히 상승
Grand View Research에 따르면, AI 로보틱스(AI Robotics) 시장은 2023년
약 127억 달러에서 2030년 약 1,247억 달러(한화 약 168조 원)로 성장하며,
연평균 38.5%의 높은 성장...
  - 문서 2: pdf/SnT GPS_290호_최종본.pdf (페이지: 3)
    내용 미리보기: 2
피지컬 AI에 대한 시장 기대도 꾸준히 상승
Grand View Research에 따르면, AI 로보틱스(AI Robotics) 시장은 2023년
약 127억 달러에서 2030년 약 1,247억 달러(한화 약 168조 원)로 성장하며,
연평균 38.5%의 높은 성장...
  - 문서 3: pdf/SnT GPS_290호_최종본.pdf (페이지: 3)
    내용 미리보기: 향후 관련 산업의 본격적인 성장과 확산이 가속화될 것으로 전망
피지컬 AI는 의료, 제조, 건설, 물류, 국방 등 다양한 산업 전반의 지능화와
자동화를 이끄는 핵심 기술로 부상하며, 국가 전략기술로서의 위상이 더욱
강화될 것으로 예상
이에 발맞춰 우리나라도 산업적 차원...
  - 문서 4: pdf/SnT GPS_290호_최종본.pdf (페이지: 3)
    내용 미리보기: 향후 관련 산업의 본격적인 성장과 확산이 가속화될 것으로 전망
피지컬 AI는 의료, 제조, 건설, 물류, 국방 등 다양한 산업 전반의 지능화와
자동화를 이끄는 핵심 기술로 부상하며, 국가 전략기술로서의 위상이 더욱
강화될 것으로 예상
이에 발맞춰 우리나라도 산업적 차원...


Question 2: 연방하원 선거 45일 후, 기독민주당(CDU)/기독사회

In [39]:
# 각 질문에 대해 3개 LLM으로 답변 생성
for q in questions:
    print(f"\n=== 질문: {q}")
    for name, chain in rag_chains.items():
        try:
            ans = chain.invoke(q)
            print(f"\n[{name}]\n{ans}")
        except Exception as e:
            print(f"\n[{name}] 실패: {e}")


['AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년에 얼마로 성장하나요?', '연방하원 선거 45일 후, 기독민주당(CDU)/기독사회당(CSU) 연합과 사민당(SPD)은 어떤 문서에 연정 합의했나요?', 'BCG는 각국 정부가 직면한 데이터 주권 및 보안 문제를 해결하고자 어떤 보고서를 발표했나요?']

=== 질문: AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년에 얼마로 성장하나요?

[GPT-3.5-turbo]
2023년 약 127억 달러에서 2030년 약 1,247억 달러로 성장할 것으로 전망됩니다. 연평균 38.5%의 높은 성장률을 기록할 것으로 예상됩니다. Grand View Research와 Statista의 예측에 따르면 AI 로보틱스 시장은 크게 성장할 것으로 전망됩니다.

[GPT-4o-mini]
AI 로보틱스(AI Robotics) 시장은 2023년 약 127억 달러에서 2030년 약 1,247억 달러로 성장할 것으로 전망됩니다. 이는 연평균 38.5%의 높은 성장률을 기록할 것으로 예상됩니다.

[Gemini-2.0-flash]
Grand View Research에 따르면, AI 로보틱스 시장은 2023년 약 127억 달러에서 2030년 약 1,247억 달러로 성장할 것으로 예상됩니다. Statista는 AI 로보틱스 시장이 2023년 약 127억 달러에서 2030년 약 643억 달러에 이를 것으로 전망합니다. 두 기관의 전망치에 차이가 있습니다.

=== 질문: 연방하원 선거 45일 후, 기독민주당(CDU)/기독사회당(CSU) 연합과 사민당(SPD)은 어떤 문서에 연정 합의했나요?

[GPT-3.5-turbo]
독일에 대한 책임(Verantwortung für Deutschland)이라는 문서에 연정 합의했습니다.

[GPT-4o-mini]
연방하원 선거 45일 후, 기독민주당(CDU)/기독사회당(CSU) 연합과 사민당(SPD)은 "독일에 대한 책임(Veran

In [58]:
# ===== 질문 리스트 =====
questions = [
    "Snowflake에 대해 설명해주세요.",
    "피지컬 AI 개발 동향에 대해 알려주세요."
]
print(questions)

['Snowflake에 대해 설명해주세요.', '피지컬 AI 개발 동향에 대해 알려주세요.']


In [59]:
# 각 질문에 대해 관련 문서 검색
for idx, question in enumerate(questions, 1):
    print(f"\n{'='*100}")
    print(f"Question {idx}: {question}")
    print(f"{'='*100}\n")
    
    # 관련 문서 검색
    relevant_docs = retriever.invoke(question)
    print(f"[검색된 관련 문서 정보]")
    print(f"총 {len(relevant_docs)}개의 문서 청크가 검색되었습니다.")
    for i, doc in enumerate(relevant_docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'Unknown')
        print(f"  - 문서 {i}: {source} (페이지: {page})")
        print(f"    내용 미리보기: {doc.page_content[:150]}...")
    print()



Question 1: Snowflake에 대해 설명해주세요.

[검색된 관련 문서 정보]
총 10개의 문서 청크가 검색되었습니다.
  - 문서 1: pdf/SnT GPS_291호_최종.pdf (페이지: 13)
    내용 미리보기: 12-데이터셋 라이브러리를 통해 단 한 줄의 코드만으로 데이터를 불러오고 
전처리할 수 있는 편의성을 제공하며 , 연구자들의 데이터 활용 진입장벽을 
획기적으로 감소
-데이터셋 카드(Dataset Card) 표준을 통해 데이터 품질 문서화와 평가 도구를  
제공함으로써...
  - 문서 2: pdf/SnT GPS_291호_최종.pdf (페이지: 13)
    내용 미리보기: 12-데이터셋 라이브러리를 통해 단 한 줄의 코드만으로 데이터를 불러오고 
전처리할 수 있는 편의성을 제공하며 , 연구자들의 데이터 활용 진입장벽을 
획기적으로 감소
-데이터셋 카드(Dataset Card) 표준을 통해 데이터 품질 문서화와 평가 도구를  
제공함으로써...
  - 문서 3: pdf/SnT GPS_291호_최종.pdf (페이지: 13)
    내용 미리보기: 12-데이터셋 라이브러리를 통해 단 한 줄의 코드만으로 데이터를 불러오고 
전처리할 수 있는 편의성을 제공하며 , 연구자들의 데이터 활용 진입장벽을 
획기적으로 감소
-데이터셋 카드(Dataset Card) 표준을 통해 데이터 품질 문서화와 평가 도구를  
제공함으로써...
  - 문서 4: pdf/SnT GPS_291호_최종.pdf (페이지: 13)
    내용 미리보기: 12-데이터셋 라이브러리를 통해 단 한 줄의 코드만으로 데이터를 불러오고 
전처리할 수 있는 편의성을 제공하며 , 연구자들의 데이터 활용 진입장벽을 
획기적으로 감소
-데이터셋 카드(Dataset Card) 표준을 통해 데이터 품질 문서화와 평가 도구를  
제공함으로써...
  - 문서 5: pdf/SnT GPS_291호_최종.pdf (페이지: 13)
    내용 미리보기: 12-데이터셋 라이브러리를 통해 단 한 줄의 

In [60]:
# 각 질문에 대해 3개 LLM으로 답변 생성
for q in questions:
    print(f"\n=== 질문: {q}")
    for name, chain in rag_chains.items():
        try:
            ans = chain.invoke(q)
            print(f"\n[{name}]\n{ans}")
        except Exception as e:
            print(f"\n[{name}] 실패: {e}")



=== 질문: Snowflake에 대해 설명해주세요.

[GPT-3.5-turbo]
Snowflake는 기업 간 안전한 데이터 교환과 AI용 데이터 마켓플레이스 역할을 수행하며, 다양한 외부 데이터를 통합하여 활용할 수 있는 편의성을 제공합니다. 또한, Snowflake는 대규모 합성 데이터셋을 출시하여 AI 학습용 데이터 유통 채널로 발전하고, 엔터프라이즈급 데이터 관리 체계를 구축하여 데이터 일관성과 품질을 유지합니다.

[GPT-4o-mini]
Snowflake는 기업 간 안전한 데이터 교환과 AI용 데이터 마켓플레이스 역할을 수행하는 플랫폼입니다. 이 마켓플레이스에는 다양한 외부 데이터가 등재되어 있어 기업들이 쉽게 통합하여 활용할 수 있습니다. 또한, ERP와 CRM 시스템의 데이터를 자동으로 연결하여 실시간 데이터 통합을 지원하고, 데이터 거버넌스 체계를 완비하여 규제 준수와 데이터 품질 관리를 동시에 달성합니다.

[Gemini-2.0-flash]
Snowflake는 기업 간 안전한 데이터 교환과 AI용 데이터 마켓플레이스 역할을 수행하는 플랫폼입니다. Snowflake 마켓플레이스에는 금융, 인구통계, 위치정보 등 수백 종의 외부 데이터가 등재되어 있어 기업들이 즉시 통합하여 활용할 수 있습니다. 또한, AI 학습용 데이터 유통 채널로 진화하고 엔터프라이즈급 데이터 관리 체계를 구축하여 AI 시대가 요구하는 데이터 거버넌스 체계를 완비했습니다.

=== 질문: 피지컬 AI 개발 동향에 대해 알려주세요.

[GPT-3.5-turbo]
피지컬 AI는 AI의 물리적 구현과 물리적 인터페이스를 통한 상호작용을 강조하는 개념으로, 학계, 연구계, 산업계에서 주목받고 있습니다. 피지컬 AI 시장은 높은 성장률을 기록하며, 미국, 중국, 일본, EU 등 주요국들 간의 경쟁이 치열하게 진행되고 있습니다. 한국도 피지컬 AI 분야에서의 기술 개발과 경쟁력 강화를 위해 노력하고 있습니다.

[GPT-4o-mini]
피지컬 AI는 AI의 물리적 구현과 실제 세계와